In [ ]:
# default_exp models.components.common

In [ ]:
# export
import torch
import torch.nn as nn
from torch.nn.utils import weight_norm

# Chomp1d

In [ ]:
# export
class Chomp1d(nn.Module):
    """
    Receives x input of dim nxcxt, and trims it so that only 
    'time available' information is used. Used for one dimensional 
    causal convolutions.
    : param chomp_size: lenght of outsample values to skip.
    """
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()

# CausalConv1D

In [ ]:
# export
ACTIVATIONS = ['ReLU',
               'Softplus',
               'Tanh',
               'SELU',
               'LeakyReLU',
               'PReLU',
               'Sigmoid']

class CausalConv1d(nn.Module):
    """
    Receives x input of dim nxcxt, computes a unidimensional 
    causal convolution.
    
    Parameters
    ----------
    in_channels: int
    out_channels: int
    activation: str 
        https://discuss.pytorch.org/t/call-activation-function-from-string
    padding: int
    kernel_size: int
    dilation: int
    
    Returns:
    x: tesor
        torch tensor of dim nxcxt 
        activation(conv1d(inputs, kernel) + bias)
    """
    def __init__(self, in_channels, out_channels, kernel_size,
                 stride, padding, dilation, activation, with_weight_norm:bool):
        super(CausalConv1d, self).__init__()
        assert activation in ACTIVATIONS, f'{activation} is not in {ACTIVATIONS}'
        
        self.conv       = nn.Conv1d(in_channels=in_channels, out_channels=out_channels, 
                                    kernel_size=kernel_size, stride=stride, padding=padding,
                                    dilation=dilation)
        if with_weight_norm: self.conv = weight_norm(self.conv)
        
        self.chomp      = Chomp1d(padding)
        self.activation = getattr(nn, activation)()
        self.causalconv = nn.Sequential(self.conv, self.chomp, self.activation)
    
    def forward(self, x):
        return self.causalconv(x)

# TimeDistributed

In [ ]:
# export
class TimeDistributed(nn.Module):
    """
    Receives x input of dim nxcxt, transforms it to txnxc
    Collapses input of dim txnxc to (txn)xc and applies to a module.
    Allows handling of variable sequence lengths and minibatch sizes.
    : param module: Module to apply input to.
    """
    def __init__(self, module):
        super(TimeDistributed, self).__init__()
        self.module = module

    def forward(self, x):
        n, c, t = x.size(0), x.size(1), x.size(2)
        x = x.permute(2, 0, 1).contiguous()
        x = x.view(t * n, -1)
        x = self.module(x)
        x = x.view(t, n, -1)
        x = x.permute(1, 2, 0).contiguous()
        return x

# FeaturewiseL1Regularizer

In [ ]:
class FeaturewiseL1Regularizer(nn.Module):
    """
    Layer meant to apply elementwise L1 regularization to a dimension.
    Receives x input of dim (nxc) and returns the input (nxc).
    """
    def __init__(self, in_features, l1_coeff=1e-3):
        super(FeaturewiseL1Regularizer).__init__()

        self.weight = nn.Parameter(t.Tensor(in_features, 1), requires_grad=True)
        #nn.init.kaiming_uniform_(self.weight, a=math.sqrt(0.5))
        nn.init.uniform_(self.weight, a=0., b=1.)
    
    def forward(self, x):
        # xt_input.size() == [batch_size, in_features]
        return x * self.weight # broadcasts batch_size